# Trading Derivatives with OKX Client Gateway

This notebook demonstrates how to trade derivatives (perpetual swaps, futures, options) using the `okx-client-gw-py` library.

## Derivatives Types

OKX offers three types of derivatives:
- **Perpetual Swaps (SWAP)**: No expiry date, funding rate mechanism
- **Futures (FUTURES)**: Fixed expiry dates (quarterly, bi-quarterly)
- **Options (OPTION)**: Call/put options on crypto assets

This tutorial focuses on perpetual swaps as they are the most commonly traded.

## Prerequisites

1. API credentials set in environment variables
2. Account mode must be one of: Single-currency margin, Multi-currency margin, or Portfolio margin
3. **Simple mode cannot trade derivatives**

## Setup and Imports

In [ ]:
from decimal import Decimal

# Core imports
from okx_client_gw import OkxHttpClient, OkxConfig
from okx_client_gw.core.auth import OkxCredentials

# Services
from okx_client_gw.application.services import (
    MarketDataService,
    InstrumentService,
    AccountService,
    TradeService,
    PublicDataService,
)

# Domain models and enums
from okx_client_gw.domain.enums import (
    InstType,
    TradeSide,
    TradeMode,
    OrderType,
    PositionSide,
    MarginMode,
)

## Configuration

In [ ]:
# Load credentials from environment variables
try:
    credentials = OkxCredentials.from_env()
    print("✓ Credentials loaded from environment")
except ValueError as e:
    print(f"⚠ No credentials found: {e}")
    print("  Derivatives trading requires authentication.")
    credentials = None

# Configure for demo trading
config = OkxConfig(use_demo=True)

## Part 1: Understanding Derivatives Markets

### Get Available Perpetual Swaps

In [ ]:
async def get_swap_instruments():
    """Get all perpetual swap instruments."""
    async with OkxHttpClient(config=config) as client:
        service = InstrumentService(client)
        instruments = await service.get_instruments(InstType.SWAP)
        
        print(f"Found {len(instruments)} perpetual swaps\n")
        
        # Show USDT-margined swaps
        usdt_swaps = [i for i in instruments if i.settle_ccy == "USDT"]
        print(f"USDT-margined swaps ({len(usdt_swaps)}):")
        for inst in usdt_swaps[:10]:
            print(f"  {inst.inst_id}: ctVal={inst.ct_val}, lever max={inst.lever}")
        print("  ...")
        
        return instruments

swap_instruments = await get_swap_instruments()

### Get Swap Market Tickers

In [ ]:
async def get_swap_tickers():
    """Get all perpetual swap tickers."""
    async with OkxHttpClient(config=config) as client:
        service = MarketDataService(client)
        tickers = await service.get_tickers(InstType.SWAP)
        
        print(f"Found {len(tickers)} swap tickers\n")
        
        # Top 5 by volume
        sorted_tickers = sorted(tickers, key=lambda t: float(t.vol_ccy_24h or 0), reverse=True)
        print("Top 5 swaps by 24h volume:")
        for ticker in sorted_tickers[:5]:
            print(f"  {ticker.inst_id}: ${float(ticker.last):,.2f} (vol: ${float(ticker.vol_ccy_24h):,.0f})")
        
        return tickers

swap_tickers = await get_swap_tickers()

### Get Funding Rates

Funding rates are periodic payments between long and short positions to keep the perpetual swap price close to the spot price.

In [ ]:
async def get_funding_info():
    """Get funding rate information for major swaps."""
    async with OkxHttpClient(config=config) as client:
        service = PublicDataService(client)
        
        swaps = ["BTC-USDT-SWAP", "ETH-USDT-SWAP", "SOL-USDT-SWAP"]
        
        print("Current Funding Rates:")
        print(f"{'Instrument':<18} {'Rate':>10} {'Annualized':>12} {'Next Funding':>20}")
        print("-" * 62)
        
        for swap in swaps:
            rate = await service.get_funding_rate(swap)
            annualized = float(rate.funding_rate) * 3 * 365 * 100
            next_time = rate.next_funding_time.strftime('%H:%M:%S') if rate.next_funding_time else "N/A"
            print(f"{swap:<18} {float(rate.funding_rate)*100:>9.4f}% {annualized:>10.2f}% {next_time:>20}")
        
        print("\n💡 Positive rate = longs pay shorts")
        print("   Negative rate = shorts pay longs")
        print("   OKX funding occurs every 8 hours (3x daily)")

await get_funding_info()

### Get Historical Funding Rates

In [ ]:
async def get_funding_history():
    """Get historical funding rates."""
    async with OkxHttpClient(config=config) as client:
        service = PublicDataService(client)
        
        rates = await service.get_funding_rate_history("BTC-USDT-SWAP", limit=10)
        
        print("BTC-USDT-SWAP Funding Rate History (last 10):")
        print(f"{'Time':>20} {'Rate':>12} {'Annualized':>12}")
        print("-" * 46)
        
        for rate in rates:
            annualized = float(rate.funding_rate) * 3 * 365 * 100
            print(f"{rate.funding_time.strftime('%Y-%m-%d %H:%M'):>20} {float(rate.funding_rate)*100:>11.4f}% {annualized:>10.2f}%")
        
        # Calculate average
        avg_rate = sum(float(r.funding_rate) for r in rates) / len(rates)
        print(f"\nAverage: {avg_rate*100:.4f}% ({avg_rate*3*365*100:.2f}% annualized)")

await get_funding_history()

## Part 2: Account Setup for Derivatives

### Check Account Configuration

Derivatives trading requires specific account modes. Simple mode cannot trade derivatives.

In [ ]:
async def check_account_mode():
    """Check account mode for derivatives eligibility."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        service = AccountService(client)
        config_data = await service.get_config()
        
        print("Account Configuration:")
        print(f"  Account Mode:   {config_data.account_mode_name}")
        print(f"  Position Mode:  {config_data.pos_mode}")
        print()
        
        can_trade_derivatives = config_data.acct_lv in ("2", "3", "4")
        
        if can_trade_derivatives:
            print("✓ This account CAN trade derivatives")
            if config_data.acct_lv == "2":
                print("  → Single-currency margin: Each position uses its own margin currency")
            elif config_data.acct_lv == "3":
                print("  → Multi-currency margin: Multiple currencies provide cross-collateral")
            elif config_data.acct_lv == "4":
                print("  → Portfolio margin: Advanced risk-based margining across positions")
        else:
            print("✗ This account CANNOT trade derivatives")
            print("  → Upgrade from Simple mode to access derivatives")
        
        return config_data

account_config = await check_account_mode()

### Get Account Balance

In [ ]:
async def get_account_balance():
    """Get account balance for derivatives trading."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        service = AccountService(client)
        balance = await service.get_balance()
        
        print("Account Balance:")
        print(f"  Total Equity:    ${float(balance.total_eq):,.2f}")
        print(f"  IMR (Initial):   ${float(balance.imr):,.2f}")
        print(f"  MMR (Maint):     ${float(balance.mmr):,.2f}")
        if balance.mgn_ratio:
            print(f"  Margin Ratio:    {float(balance.mgn_ratio) * 100:.1f}%")
        print(f"  Account Health:  {'✓ Healthy' if balance.is_healthy else '⚠ Low margin'}")
        
        return balance

balance = await get_account_balance()

### Set Leverage

Configure leverage for derivatives trading. Higher leverage = higher risk.

In [ ]:
async def set_leverage_examples():
    """Examples of setting leverage for different scenarios."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        service = AccountService(client)
        
        # Example 1: Set leverage for cross-margin positions
        print("Setting 5x leverage for BTC-USDT-SWAP (cross margin):")
        result = await service.set_leverage(
            inst_id="BTC-USDT-SWAP",
            lever=5,
            mgn_mode=MarginMode.CROSS,
        )
        print(f"  Result: {result}")
        
        # Example 2: Set leverage for isolated-margin positions
        print("\nSetting 10x leverage for BTC-USDT-SWAP (isolated margin):")
        result = await service.set_leverage(
            inst_id="BTC-USDT-SWAP",
            lever=10,
            mgn_mode=MarginMode.ISOLATED,
        )
        print(f"  Result: {result}")
        
        return result

# Uncomment to execute:
# leverage_result = await set_leverage_examples()

### Set Position Mode

Choose between:
- **Long/Short mode**: Separate positions for long and short
- **Net mode**: Single net position

In [ ]:
async def set_position_mode(mode: str):
    """Set position mode (long_short_mode or net_mode)."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        service = AccountService(client)
        result = await service.set_position_mode(mode)
        
        print(f"Position mode set to: {mode}")
        print(f"Result: {result}")
        
        if mode == "long_short_mode":
            print("\n💡 In long/short mode:")
            print("   - You can hold both long AND short positions simultaneously")
            print("   - Use posSide='long' or posSide='short' when placing orders")
        else:
            print("\n💡 In net mode:")
            print("   - You have a single net position")
            print("   - Buy increases position, sell decreases it")
        
        return result

# Uncomment to execute:
# await set_position_mode("long_short_mode")
# await set_position_mode("net_mode")

## Part 3: Trading Perpetual Swaps

### Place a Limit Order (Isolated Margin, Net Mode)

In [ ]:
async def place_swap_limit_order():
    """Place a limit order on a perpetual swap."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        # Get current price
        market_service = MarketDataService(client)
        ticker = await market_service.get_ticker("BTC-USDT-SWAP")
        current_price = float(ticker.last)
        
        # Place limit order 5% below market (won't fill immediately)
        limit_price = Decimal(str(round(current_price * 0.95, 1)))
        
        trade_service = TradeService(client)
        result = await trade_service.place_limit_order(
            inst_id="BTC-USDT-SWAP",
            side=TradeSide.BUY,
            sz=Decimal("1"),  # 1 contract
            px=limit_price,
            td_mode=TradeMode.ISOLATED,
            pos_side=PositionSide.NET,  # For net mode
        )
        
        if result.get("sCode") == "0":
            print("✓ Swap limit order placed successfully!")
            print(f"  Order ID:    {result.get('ordId')}")
            print(f"  Instrument:  BTC-USDT-SWAP")
            print(f"  Side:        BUY (Long)")
            print(f"  Price:       ${float(limit_price):,.1f}")
            print(f"  Size:        1 contract")
            print(f"  Mode:        Isolated margin, net position")
        else:
            print(f"✗ Order failed: {result.get('sMsg')}")
        
        return result

# Uncomment to execute:
# swap_order = await place_swap_limit_order()

### Place a Market Order

In [ ]:
async def place_swap_market_order():
    """Place a market order on a perpetual swap."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        trade_service = TradeService(client)
        
        result = await trade_service.place_market_order(
            inst_id="BTC-USDT-SWAP",
            side=TradeSide.BUY,
            sz=Decimal("1"),  # 1 contract
            td_mode=TradeMode.ISOLATED,
            pos_side=PositionSide.NET,
        )
        
        if result.get("sCode") == "0":
            print("✓ Swap market order placed successfully!")
            print(f"  Order ID: {result.get('ordId')}")
        else:
            print(f"✗ Order failed: {result.get('sMsg')}")
        
        return result

# Uncomment to execute:
# market_order = await place_swap_market_order()

### Place Orders in Long/Short Mode

When using `long_short_mode`, you must specify `pos_side` as `LONG` or `SHORT`.

In [ ]:
async def place_long_short_orders():
    """Place orders in long/short position mode."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        market_service = MarketDataService(client)
        ticker = await market_service.get_ticker("BTC-USDT-SWAP")
        current_price = float(ticker.last)
        
        trade_service = TradeService(client)
        
        # Open a long position (buy to open long)
        print("Opening LONG position:")
        long_result = await trade_service.place_limit_order(
            inst_id="BTC-USDT-SWAP",
            side=TradeSide.BUY,
            sz=Decimal("1"),
            px=Decimal(str(round(current_price * 0.95, 1))),
            td_mode=TradeMode.ISOLATED,
            pos_side=PositionSide.LONG,  # Explicitly long
        )
        print(f"  Order ID: {long_result.get('ordId')} - {long_result.get('sMsg', 'OK')}")
        
        # Open a short position (sell to open short)
        print("\nOpening SHORT position:")
        short_result = await trade_service.place_limit_order(
            inst_id="BTC-USDT-SWAP",
            side=TradeSide.SELL,
            sz=Decimal("1"),
            px=Decimal(str(round(current_price * 1.05, 1))),
            td_mode=TradeMode.ISOLATED,
            pos_side=PositionSide.SHORT,  # Explicitly short
        )
        print(f"  Order ID: {short_result.get('ordId')} - {short_result.get('sMsg', 'OK')}")
        
        print("\n💡 In long/short mode:")
        print("   - BUY + LONG = Open long position")
        print("   - SELL + LONG = Close long position")
        print("   - SELL + SHORT = Open short position")
        print("   - BUY + SHORT = Close short position")
        
        return long_result, short_result

# Uncomment to execute (ensure position mode is set to long_short_mode first):
# long_short_orders = await place_long_short_orders()

### Get Open Positions

In [ ]:
async def get_positions():
    """Get current open positions."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        service = AccountService(client)
        positions = await service.get_positions(inst_type=InstType.SWAP)
        
        if positions:
            print(f"Open Positions ({len(positions)}):")
            print(f"{'Instrument':<18} {'Side':>6} {'Size':>10} {'Entry':>12} {'Mark':>12} {'UPL':>12} {'Liq':>12}")
            print("-" * 86)
            
            for pos in positions:
                side = "LONG" if pos.is_long else "SHORT" if pos.is_short else "NET"
                upl_color = "" if float(pos.upl) >= 0 else "-"
                liq = f"${float(pos.liq_px):,.0f}" if pos.liq_px else "N/A"
                print(f"{pos.inst_id:<18} {side:>6} {float(pos.pos):>10.4f} ${float(pos.avg_px):>10,.1f} ${float(pos.mark_px):>10,.1f} ${float(pos.upl):>10,.2f} {liq:>12}")
            
            # Summary
            total_upl = sum(float(p.upl) for p in positions)
            print(f"\nTotal UPL: ${total_upl:,.2f}")
        else:
            print("No open positions")
        
        return positions

positions = await get_positions()

### Close a Position

In [ ]:
async def close_position(inst_id: str, pos_side: PositionSide = PositionSide.NET):
    """Close an open position with a market order."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        account_service = AccountService(client)
        
        # Get current position
        positions = await account_service.get_positions(inst_id=inst_id)
        if not positions:
            print(f"No position found for {inst_id}")
            return None
        
        pos = positions[0]
        
        # Determine close direction
        if pos.is_long:
            close_side = TradeSide.SELL
        else:
            close_side = TradeSide.BUY
        
        trade_service = TradeService(client)
        result = await trade_service.place_market_order(
            inst_id=inst_id,
            side=close_side,
            sz=abs(pos.pos),
            td_mode=TradeMode.ISOLATED,
            pos_side=pos_side,
            reduce_only=True,  # Ensure we only close, not open new position
        )
        
        if result.get("sCode") == "0":
            print(f"✓ Position closed: {inst_id}")
            print(f"  Closed {abs(float(pos.pos))} contracts at market")
        else:
            print(f"✗ Close failed: {result.get('sMsg')}")
        
        return result

# Uncomment to execute:
# close_result = await close_position("BTC-USDT-SWAP")

### Cancel and Amend Orders

In [ ]:
async def manage_swap_orders():
    """Get, amend, and cancel swap orders."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        trade_service = TradeService(client)
        
        # Get pending orders
        pending = await trade_service.get_pending_orders(inst_type=InstType.SWAP)
        
        if pending:
            print(f"Pending Swap Orders ({len(pending)}):")
            for order in pending:
                print(f"  {order.ord_id}: {order.inst_id} {order.side.value} {order.sz} @ {order.px}")
            
            # Example: Amend first order
            # first_order = pending[0]
            # await trade_service.amend_order(
            #     inst_id=first_order.inst_id,
            #     ord_id=first_order.ord_id,
            #     new_sz="2",  # Change size to 2 contracts
            # )
            
            # Example: Cancel first order
            # await trade_service.cancel_order(
            #     inst_id=first_order.inst_id,
            #     ord_id=first_order.ord_id,
            # )
        else:
            print("No pending swap orders")
        
        return pending

swap_orders = await manage_swap_orders()

### Get Order History for Swaps

In [ ]:
async def get_swap_order_history():
    """Get order history for perpetual swaps."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return None
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        trade_service = TradeService(client)
        orders = await trade_service.get_order_history(
            inst_type=InstType.SWAP,
            limit=10
        )
        
        if orders:
            print(f"Swap Order History (last {len(orders)}):")
            for order in orders:
                status = "✓" if order.state.value == "filled" else "✗" if order.state.value == "canceled" else "○"
                print(f"  {status} {order.c_time.strftime('%m-%d %H:%M')} {order.inst_id} {order.side.value} {order.sz} @ {order.px or 'mkt'} [{order.state.value}]")
        else:
            print("No swap order history")
        
        return orders

history = await get_swap_order_history()

## Part 4: Cross-Margin vs Isolated-Margin

### Understanding Margin Modes

- **Cross Margin**: All positions share the account's total equity as collateral
- **Isolated Margin**: Each position has its own dedicated margin

In [ ]:
async def compare_margin_modes():
    """Compare cross vs isolated margin orders."""
    if not credentials:
        print("⚠ Skipping: No credentials available")
        return
    
    async with OkxHttpClient(config=config, credentials=credentials) as client:
        market_service = MarketDataService(client)
        ticker = await market_service.get_ticker("BTC-USDT-SWAP")
        price = Decimal(str(round(float(ticker.last) * 0.95, 1)))
        
        trade_service = TradeService(client)
        
        print("Cross Margin Order:")
        print("  - Shares margin with all cross positions")
        print("  - Lower liquidation risk (more collateral)")
        print("  - But losses in one position affect all positions")
        print()
        
        # Cross margin order (commented out)
        # cross_result = await trade_service.place_limit_order(
        #     inst_id="BTC-USDT-SWAP",
        #     side=TradeSide.BUY,
        #     sz=Decimal("1"),
        #     px=price,
        #     td_mode=TradeMode.CROSS,  # Cross margin
        #     pos_side=PositionSide.NET,
        # )
        
        print("Isolated Margin Order:")
        print("  - Each position has dedicated margin")
        print("  - Higher liquidation risk per position")
        print("  - But losses limited to position's margin")
        print()
        
        # Isolated margin order (commented out)
        # isolated_result = await trade_service.place_limit_order(
        #     inst_id="BTC-USDT-SWAP",
        #     side=TradeSide.BUY,
        #     sz=Decimal("1"),
        #     px=price,
        #     td_mode=TradeMode.ISOLATED,  # Isolated margin
        #     pos_side=PositionSide.NET,
        # )
        
        print("💡 Choose based on your risk management strategy")

await compare_margin_modes()

## Summary

This notebook covered derivatives trading with `okx-client-gw-py`:

**Market Data**:
- Get swap instruments and tickers
- Funding rates (current and historical)

**Account Setup**:
- Check account mode eligibility
- Set leverage and position mode

**Trading**:
- Place limit and market orders
- Net mode vs long/short mode
- Cross margin vs isolated margin
- Position management and closing

**Key Concepts**:
- Simple mode cannot trade derivatives
- Position mode affects how you specify orders
- Leverage amplifies both gains and losses
- Funding rates affect long-term holding costs

For market data exploration and analysis, see `03_market_data_exploration.ipynb`.